# Load User Defined Data into ApertureDB
The following notebook explains how to use ApertureDB's user friendly Python wrappers to add data to the DB.

In [ ]:
#Create connections and clean up.

from aperturedb.Utils import Utils, create_connector


db = create_connector()
utils = Utils(db)

print(utils.remove_all_objects())


## Steps

### Declare the user defined models

User models are a declarative way to define what objects the user's application will deal with. These are built on top of pydantic's models.

There are certain specific subclasses that are meant to be used with ApertureDB's built-in objects for example:

- IdentityDataModel
- ImageDataModel
- VideoDataModel

In [1]:
from typing import List
from enum import Enum

from aperturedb.Query import generate_save_query
from aperturedb.DataModels import ImageDataModel, IdentityDataModel
import random

class Side(Enum):
    RIGHT = 1
    LEFT = 2

class Finger(IdentityDataModel):
    nail_clean: bool = False

class Hand(ImageDataModel):
    side: Side = None
    thumb: Finger = None
    fingers: List[Finger] = []

class Person(IdentityDataModel):
    name: str =  ""
    hands : List[Hand] = []
    dominant_hand : Hand = None

# Helper function to create a Hand with attached fingers.
def make_hand(side: Side) -> Hand:
    # The field 'file' is enforced by the ImageModel.
    hand = Hand(side = side, url= "../similarity_search/bruce-lee.jpg")

    hand.fingers = [Finger(nail_clean=True) if random.randint(0, 1) == 1  else Finger(nail_clean=False) for i in range(5)]
    # thumb is a finger. When persisted in ApertureDB, it should not create a new entity but add a new connection to the same source
    # destination node. This also saves a redundant blob upload.
    hand.thumb = hand.fingers[0]
    return hand


### Create objects from the defined models

Let's create some people with the above schema.

In [2]:
people = []

for i in range(10):
    person = Person(name=f"apertureUser{i+1}")
    left_hand = make_hand(Side.LEFT)
    right_hand = make_hand(Side.RIGHT)
    person.hands.extend([left_hand, right_hand])
    person.dominant_hand = person.hands[0]
    people.append(person)


### Persist the objects in ApertureDB

Without needing to write any queries!

In [ ]:


for person in people:
    q, b, count = generate_save_query(person)
    r, _ = db.query(q, b)

    if not db.last_query_ok():
        import json
        db.print_last_response()
        print(json.dumps(q))

In [ ]:
from devtools import debug

# We created a query with 31 commands
debug(len(q))

connections = list(filter(lambda x: "AddConnection" in x, q))
#The number of connections is 15. (3 for person to hands, 10 + 2 for hands to fingers)
debug(len(connections))

#The number of Entities added is 13 (12 for fingers, 1 for person)
nodes = list(filter(lambda x: "AddEntity" in x, q))
debug(len(nodes))

#The number of Images added is 2 (2 hands)
nodes = list(filter(lambda x: "AddImage" in x, q))
debug(len(nodes))

#The number of Images refered is 1 (1 dominnat hand)
nodes = list(filter(lambda x: "FindImage" in x, q))
debug(len(nodes))

# The entire BOM is 31 = 15 + 13 + 2 + 1